In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re  # regex para validar si existe subfijos
import numpy as np

# Función para la corrección de Shirley
def shirley_correction(raman_shift, intensity):
    """
    Aplica la corrección de Shirley para un espectro.
    
    Parameters:
        raman_shift (array-like): Valores del eje X (Raman Shift).
        intensity (array-like): Intensidades del espectro (eje Y).
        
    Returns:
        corrected_intensity (array-like): Intensidades corregidas.
    """
    corrected_intensity = intensity.copy()
    start = corrected_intensity[0]
    end = corrected_intensity[-1]
    
    for _ in range(100):  # Máximo 100 iteraciones
        background = start + (end - start) * np.cumsum(corrected_intensity) / np.sum(corrected_intensity)
        corrected_intensity = intensity - background
        corrected_intensity[corrected_intensity < 0] = 0  # Evitar valores negativos
    
    return corrected_intensity

# Lee el archivo CSV
df = pd.read_csv('limpio.csv', delimiter=',')

# Muestra las primeras filas del DataFrame para verificar
print(df.head())
if any(re.search(r'\.\d+$', col) for col in df.columns):
    # Si hay columnas con sufijos, eliminarlos
    df.columns = [re.sub(r'\.\d+$', '', col) for col in df.columns]
    print("Se eliminaron los sufijos numéricos de los encabezados.")
# Muestra las primeras filas del DataFrame para verificar
print(df.head())

# Aplicar la corrección de Shirley a cada espectro
raman_shift = df['Ramanshift']
corrected_spectra = {}

for col in df.columns[1:]:  # Excluir la columna 'Ramanshift'
    intensity = df[col].values
    corrected_spectra[col] = shirley_correction(raman_shift, intensity)

# Crear un DataFrame con los datos corregidos
df_corrected = pd.DataFrame(corrected_spectra)
df_corrected.insert(0, 'Ramanshift', raman_shift)

# Identificar los tipos únicos de valores en los encabezados
unique_types = set(col for col in df_corrected.columns if col != "Ramanshift")

# Colores para cada tipo
colors = plt.cm.tab20.colors  # Una paleta de colores suficientemente grande
color_map = {unique: colors[i % len(colors)] for i, unique in enumerate(unique_types)}

# Graficar cada tipo una sola vez en la leyenda
plt.figure(figsize=(14, 10))

for unique_type in unique_types:
    # Filtrar las columnas correspondientes al tipo actual
    columns = [col for col in df_corrected.columns if col.startswith(unique_type)]
    
    # Graficar todas las columnas del tipo actual
    for col in columns:
        plt.plot(df_corrected['Ramanshift'], df_corrected[col], color=color_map[unique_type], alpha=0.6)
    
    # Agregar una entrada en la leyenda solo para el tipo (una vez)
    plt.plot([], [], label=unique_type, color=color_map[unique_type])  # Dummy plot for legend

# Etiquetas y leyendas
plt.title("Espectros Raman Corregidos (Shirley)", fontsize=16)
plt.xlabel("Raman Shift (cm⁻¹)", fontsize=14)
plt.ylabel("Intensidad Corregida", fontsize=14)
plt.legend(title="Tipos", fontsize=12, loc='upper right', frameon=False)
plt.grid(True)

# Mostrar la gráfica
plt.show()
